In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from huggingface_hub import login

HF_Token = "your own key"
login(token=HF_Token)

In [ ]:
!pip install transformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 32.5 MB/s eta 0:00:00


# load the tokenizer and model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer_3B = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct"
)
tokenizer_3B.pad_token = tokenizer_3B.eos_token

Llama_3B_Instruct = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
Llama_3B_Instruct

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

# load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install datasets

In [ ]:
from datasets import load_dataset

# Load json data from disk and sample subset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/chatbot/sft/sft_dataset.json")

# Sample desired number of samples (adjust this number as needed)
SAMPLE_SIZE = 6000
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(min(SAMPLE_SIZE, len(dataset["train"]))))

print(f"Using {len(dataset['train'])} samples for training")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Using 6000 samples for training
DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 6000
    })
})


In [ ]:
dataset["train"]

Dataset({
    features: ['messages'],
    num_rows: 6000
})

# PEFT and SFTTrainer

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
  lora_alpha=16,
  r=8,
  lora_dropout=0.1,
  bias="none",
  target_modules=["q_proj", "v_proj"],
  task_type="CAUSAL_LM",
)

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from trl import SFTConfig

SFT_config = SFTConfig(
    output_dir="/content/drive/MyDrive/chatbot/sft/Llama-3.2-3B",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=3e-4,
    lr_scheduler_type="cosine",
    warmup_steps=50,
    weight_decay=0.01,
    optim="adamw_torch_fused",
    max_length=512,
    report_to="tensorboard",
    logging_steps=25,
    save_steps=500,
    dataloader_num_workers=4,
    bf16=True,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    disable_tqdm=False,
    log_level="info",
    logging_first_step=True,
)

In [ ]:
from trl import SFTTrainer

SFT_trainer = SFTTrainer(
    model=Llama_3B_Instruct,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    args=SFT_config
)


Tokenizing train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Using auto half precision backend


In [ ]:
SFT_trainer.train()

***** Running training *****
  Num examples = 6,000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2,250
  Number of trainable parameters = 2,293,760
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,3.723200
25,3.439700
50,2.354900
75,1.953000
100,1.934900
125,1.934300
150,1.942300
175,1.926600
200,1.861300
225,1.894600


Saving model checkpoint to /content/drive/MyDrive/chatbot/sft/Llama-3.2-3B/checkpoint-500
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3

TrainOutput(global_step=2250, training_loss=1.775143954065111, metrics={'train_runtime': 1287.3375, 'train_samples_per_second': 13.982, 'train_steps_per_second': 1.748, 'total_flos': 1.3557097775662694e+17, 'train_loss': 1.775143954065111})

In [ ]:
from peft import AutoPeftModelForCausalLM
SFT_llama_3B = AutoPeftModelForCausalLM.from_pretrained("/content/drive/MyDrive/chatbot/sft/Llama-3.2-3B/checkpoint-2000", torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
SFT_llama_3B

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feat

In [ ]:
SFT_llama_3B = SFT_llama_3B.merge_and_unload()

In [ ]:
SFT_llama_3B.save_pretrained("/content/drive/MyDrive/chatbot/sft/Llama-3.2-3B/merged_checkpoint", safe_serialization=True)

# test

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

sft_model = AutoModelForCausalLM.from_pretrained(
    "./Llama-3.2-3B/merged_checkpoint",
    low_cpu_mem_usage=True,
    device_map="auto",
    torch_dtype=torch.float16,
)

C:\Users\23104\miniconda3\envs\PythonProject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.87s/it]


In [14]:
import torch
from transformers import pipeline

# Build once and reuse to avoid reloading the model each call
def build_pipe(model, tokenizer):
    return pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,   # change to float16 if needed
        device_map="auto",
    )

def chat_generate(pipe, tokenizer, user_text, system_text=None):
    # 1) Prepare chat messages (system first, then user)
    messages = []
    if system_text:
        messages.append({"role": "system", "content": system_text})
    messages.append({"role": "user", "content": user_text})

    # 2) Render messages with the model's chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True  # adds assistant header to start generation
    )

    # 3) Use both EOS and EOT if available for natural stop
    try:
        eot_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")
        if eot_id is not None and eot_id != tokenizer.unk_token_id:
            eos_ids = [tokenizer.eos_token_id, eot_id]
        else:
            eos_ids = tokenizer.eos_token_id
    except Exception:
        eos_ids = tokenizer.eos_token_id

    # 4) Generate (return only the new text)
    out = pipe(
        prompt,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.95,     # slightly higher temp helps produce emojis
        top_p=0.95,
        repetition_penalty=1.03,
        eos_token_id=eos_ids,
        return_full_text=False
    )

    # 5) text-generation pipeline returns plain string
    return out[0]["generated_text"]


In [23]:
pipe = build_pipe(sft_model, tokenizer)
result = chat_generate(
    pipe, tokenizer,
    user_text="Hello, I am concerned about my diabetes.",
)
print(result)

Device set to use cuda:0


Welcome to Chat Doctor, There are three types of diabetes1. Type1 or immune mediated diabetes2. Insulin resistance &3. Gestational Diabetes. Treatment of this depends upon the underlying cause. If you have Type 1 diabetes the medicine you require is insulin, if you have insulin resistance you will require diet and physical exercises and if your diabetes is in pregnancy then it will require diet and monitor the blood sugar levels by monitoring the urine and urine sugar. Get well soon.
